<a href="https://colab.research.google.com/github/aletbm/MySolutions_MLZoomcamp2024_DataTalks.Club/blob/main/03_Machine_Learning_for_Classification/3_Machine_Learning_for_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3rd week's homework - by [Alexander Daniel Rios](https://linktr.ee/aletbm)

In [130]:
!wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

--2024-10-15 01:20:49--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank+marketing.zip.4’

bank+marketing.zip.     [    <=>             ] 999.85K  1.08MB/s    in 0.9s    

2024-10-15 01:20:51 (1.08 MB/s) - ‘bank+marketing.zip.4’ saved [1023843]



In [131]:
!unzip -o bank+marketing.zip
!unzip -o bank-additional.zip
!unzip -o bank.zip

Archive:  bank+marketing.zip
 extracting: bank.zip                
 extracting: bank-additional.zip     
Archive:  bank-additional.zip
  inflating: bank-additional/.DS_Store  
  inflating: __MACOSX/bank-additional/._.DS_Store  
  inflating: bank-additional/.Rhistory  
  inflating: bank-additional/bank-additional-full.csv  
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: __MACOSX/._bank-additional  
Archive:  bank.zip
  inflating: bank-full.csv           
  inflating: bank-names.txt          
  inflating: bank.csv                


In [132]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [133]:
df = pd.read_csv("bank-full.csv", delimiter=';')
df.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


# Features
For the rest of the homework, you'll need to use only these columns:

+ age,
+ job,
+ marital,
+ education,
+ balance,
+ housing,
+ contact,
+ day,
+ month,
+ duration,
+ campaign,
+ pdays,
+ previous,
+ poutcome,
+ y

# Data preparation
Select only the features from above.

Check if the missing values are presented in the features.

In [134]:
df = df.loc[:, ~df.columns.isin(['duration', 'loan'])].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   contact    45211 non-null  object
 8   day        45211 non-null  int64 
 9   month      45211 non-null  object
 10  campaign   45211 non-null  int64 
 11  pdays      45211 non-null  int64 
 12  previous   45211 non-null  int64 
 13  poutcome   45211 non-null  object
 14  y          45211 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.2+ MB


In [135]:
df.isna().sum()

,0
age,0
job,0
marital,0
education,0
default,0
balance,0
housing,0
contact,0
day,0
month,0


# Question 1
What is the most frequent observation (mode) for the column education?

+ `unknown`
+ `primary`
+ `secondary`
+ `tertiary`

In [136]:
df.education.mode()[0]

'secondary'

# Question 2
Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

+ `age` and `balance`
+ `day` and `campaign`
+ `day` and `pdays`
+ `pdays` and `previous`

In [137]:
corr = df.corr(numeric_only=True).unstack().reset_index().rename(columns={0:"corr"})
corr = corr[corr['corr'] != 1].copy()
corr.sort_values(by='corr', ascending=False)

,level_0,level_1,corr
34,previous,pdays,0.454820
29,pdays,previous,0.454820
20,campaign,day,0.162490
15,day,campaign,0.162490
6,balance,age,0.097783
1,age,balance,0.097783
31,previous,balance,0.016674
11,balance,previous,0.016674
18,campaign,age,0.004760
3,age,campaign,0.004760


# Target encoding
Now we want to encode the y variable.

Let's replace the values yes/no with 1/0.

In [138]:
df['y'].unique()

array(['no', 'yes'], dtype=object)

In [139]:
df.y = df.y.map({"no":0, "yes":1}).astype('int')
df.y.unique()

array([0, 1])

#Split the data
+ Split your data in train/val/test sets with 60%/20%/20% distribution.
+ Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
+ Make sure that the target value y is not in your dataframe.

In [140]:
seed = 42
df_X, df_y = df.drop(['y'], axis=1).copy(), df['y'].copy()

X_train, X_val, y_train, y_val = train_test_split(df_X, df_y, train_size=0.6, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, train_size=0.5, random_state=seed)

# Question 3
+ Calculate the mutual information score between `y` and other categorical variables in the dataset. Use the training set only.
+ Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?

+ `contact`
+ `education`
+ `housing`
+ `poutcome`

In [141]:
def MI_score(X, y):
 return mutual_info_score(X, y_train)

X_train.select_dtypes('object').apply(lambda x: round(MI_score(x, y_train), 2)).sort_values(ascending=False).to_frame(name='MI')

,MI
poutcome,0.03
month,0.02
job,0.01
housing,0.01
contact,0.01
marital,0.00
education,0.00
default,0.00


# One-Hot Encoding

In [142]:
train_dicts = X_train.to_dict(orient='records')
val_dicts = X_val.to_dict(orient='records')
test_dicts = X_test.to_dict(orient='records')

dv = DictVectorizer(sparse=False).set_output(transform='pandas').fit(train_dicts)

X_train = dv.transform(train_dicts)
X_val = dv.transform(val_dicts)
X_test = dv.transform(test_dicts)

X_train.head(5)

,age,balance,campaign,contact=cellular,contact=telephone,contact=unknown,day,default=no,default=yes,education=primary,...,month=may,month=nov,month=oct,month=sep,pdays,poutcome=failure,poutcome=other,poutcome=success,poutcome=unknown,previous
0,45.0,-100.0,6.0,0.0,0.0,1.0,27.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0
1,29.0,166.0,8.0,1.0,0.0,0.0,28.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0
2,31.0,121.0,1.0,0.0,0.0,1.0,20.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0
3,40.0,1693.0,1.0,1.0,0.0,0.0,17.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0
4,28.0,317.0,3.0,0.0,0.0,1.0,16.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0


# Question 4
+ Now let's train a logistic regression.
+ Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
+ Fit the model on the training dataset.
+ To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
+ `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
+ Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

+ 0.6
+ 0.7
+ 0.8
+ 0.9

In [143]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=seed)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [144]:
y_pred_val = model.predict_proba(X_val)[:, 1] >= 0.5
acc = (y_val == y_pred_val).mean()
round(acc, 2)

0.89

#Question 5
+ Let's find the least useful feature using the feature elimination technique.
+ Train a model with all these features (using the same parameters as in Q4).
+ Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
+ For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

+ `age`
+ `balance`
+ `marital`
+ `previous`

> Note: The difference doesn't have to be positive.

In [145]:
features_list = ["age", "balance", "marital", "previous"]
acc_features = []
for feature in features_list:
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=seed)
    mask = ~X_train.columns.str.contains(feature)
    model.fit(X_train.loc[:, mask], y_train)
    y_pred_val = model.predict_proba(X_val.loc[:, mask])[:, 1] >= 0.5
    acc_features.append((y_val == y_pred_val).mean())

pd.DataFrame(data={'accuracy':acc-acc_features}, index=[f"without_{feature}" for feature in features_list]).sort_values(by='accuracy', ascending=False)

,accuracy
without_age,0.000000
without_marital,-0.000442
without_balance,-0.000553
without_previous,-0.001548


# Question 6
+ Now let's train a regularized logistic regression.
+ Let's try the following values of the parameter `C`: `[0.01, 0.1, 1, 10, 100]`.
+ Train models using all the features as in Q4.
+ Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these `C` leads to the best accuracy on the validation set?

+ 0.01
+ 0.1
+ 1
+ 10
+ 100

>Note: If there are multiple options, select the smallest C.

In [146]:
C_list = [0.01, 0.1, 1, 10, 100]
acc_C = []
for C in C_list:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=seed)
    model.fit(X_train, y_train)
    y_pred_val = model.predict_proba(X_val)[:, 1] >= 0.5
    acc_C.append(round((y_val == y_pred_val).mean(), 3))

pd.DataFrame(data={'C':C_list, 'accuracy':acc_C}).sort_values(by='accuracy', ascending=False)

,C,accuracy
3,10.00,0.895
0,0.01,0.894
1,0.10,0.894
2,1.00,0.894
4,100.00,0.894
